# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [30]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [31]:
#Checking our current working directory
print(os.getcwd())

#Getting our current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

#Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
#Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [32]:
#Initiating an empty list of rows that will be generated from each file

full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
#Extracting each data row one by one and append it    

        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
print(full_data_rows_list)

#Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88'], ['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88'], ["The B-52's", 'Logged In', 'Gianna', 'F', '0', 'Jones', '321.54077', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54087E+12', '107', 'Love Shack', '200', '1.54328E+12', '38'], ['Die Mooskirc

In [33]:
# checking the number of rows in our csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

Creating a Cluster

In [34]:
#Creating a connection to a Cassandra instance your local machine(127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

#Creating a session to establish connection and begin executing queries
session = cluster.connect()

Creating a Keyspace


In [36]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

Setting KEYSPACE to the keyspace specified above

In [38]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Creating the table song_bysession_byitem that we will use to execute our first query: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [39]:
try:
    session.execute("CREATE TABLE IF NOT EXISTS song_bysession_byitem (artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession)); ")
except Exception as e:
    print(e)

    





We are selecting the columns from the table that were specifically mentioned at the first query and we are using a composite primary key that will uniquely identify each row, based on the query. The sessionId is used as the partition key, which is the primary lookup to find a set of rows in our table, and the iteminSession is used as the clustering key, which is used to define the ordering within the partition

In [40]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader: 
#Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_bysession_byitem(artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
#Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [41]:
#Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

question1 = "select artist, song, length from song_bysession_byitem WHERE sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(question1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


Creating the table song_byuser_bysession that we will use to execute our second query: Give me the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [42]:

try:
   session.execute("CREATE TABLE IF NOT EXISTS song_byuser_bysession (artist text, song text, itemInSession int, firstName text, lastName text, userId int, sessionId int, PRIMARY KEY ((userId, sessionId), itemInSession)); ")

except Exception as e:
    print(e)

We are selecting the columns from the table that were specifically mentioned at the second query and we are using a composite primary key that will uniquely identify each row based on the query(here we are using userId and sessionId as a composite partition key, which is the primary lookup to find a set of rows in our table, and the itemInSession as a clustering key, to define the ordering within the partition)

In [43]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:
#Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_byuser_bysession(artist, song, itemInSession, firstName, lastName, userId, sessionId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
#Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))


In [44]:
#Query 2: Give me the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

question2 = "select artist, song, firstName, lastName from song_byuser_bysession WHERE userId = 10 and sessionId = 182"

try:
    rows = session.execute(question2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)                   

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


Creating the table user_bysong that we will use to execute our third query: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [45]:
try:
   session.execute("CREATE TABLE IF NOT EXISTS user_bysong (firstName text, lastName text, song text, userId int, PRIMARY KEY (song, userId)); ")

except Exception as e:
    print(e)               

We are selecting the columns from the table that were specifically mentioned at the third query and we are using a composite primary key that will uniquely identify each row based on the query(here we are using song as a partition key, which is the primary lookup to find a set of rows in our table, and userId as a clustering key, to define the ordering within the partition.

In [46]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    
    for line in csvreader:      
#Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO user_bysong(firstName, lastName, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
#Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[1], line[4], line[9], int(line[10]) ))


In [47]:
#Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

question3 = "select firstName, lastName from user_bysong WHERE song = 'All Hands Against His Own' "

try:
    rows = session.execute(question3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname) 

Jacqueline Lynch
Sara Johnson
Tegan Levine


Dropping the tables before closing out the sessions

In [48]:

try:
    session.execute("DROP TABLE song_bysession_byitem")
    session.execute("DROP TABLE song_byuser_bysession")
    session.execute("DROP TABLE user_bysong")
    
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()